In [33]:
# Import modules

import os
import pickle
import random 
import operator
import math
import numpy as np
import pandas as pd
from pathlib import Path
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from time import gmtime, strftime

/Users/danielokeeffe/opt/miniconda3/envs/basic_ml_37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [24]:
data_path = '../../data/genres/'

In [25]:
feature_store_path = '../../features'

In [29]:
# Generate the current time for tagging files. Only relevant if the file is to be saved.
current_time = strftime('%Y-%m-%d_%H%M%S', gmtime())

In [27]:
if feature_store_path is not None:
    Path(feature_store_path).mkdir(parents=True, exist_ok=True)

In [44]:
# How do I load this thing? Each genre seems to have a subfolder containing the soundclips.
i = 1
if feature_store_path is not None:
    print('Saving sound file features in directory: {0}'.format(feature_store_path))
    f = open(os.path.join(feature_store_path, 'features_{0}.dat'.format(current_time)), 'wb')
else:
    print('Saving sound file features in current working directory: {0}'.format(os.getcwd()))
    f = open('features_{0}.dat'.format(current_time), 'wb')
class_encoding_dict_ = {}
for folder in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, folder)):
        for f_ in os.listdir(os.path.join(data_path, folder)):
            (rate, sig) = wav.read(os.path.join(os.path.join(data_path, folder), f_))
            mfcc_feat = mfcc(sig, rate, winlen=0.02, appendEnergy=False)
            covariance_ = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature_ = (mean_matrix, covariance_, i)
            pickle.dump(feature_, f)
        class_encoding_dict_[i] = folder
        i += 1
f.close()

Saving sound file features in directory: ../../features


In [45]:
class_encoding_dict_

{1: 'pop',
 2: 'metal',
 3: 'disco',
 4: 'blues',
 5: 'reggae',
 6: 'classical',
 7: 'rock',
 8: 'hiphop',
 9: 'country',
 10: 'jazz'}

In [46]:
# And save this to a DataFrame so we know what the class encodings are:
df_class_encodings = pd.DataFrame.from_dict(class_encoding_dict_, orient='index').reset_index()

In [47]:
df_class_encodings.columns = ['encoding', 'class_name']

In [48]:
df_class_encodings

,encoding,class_name
0,1,pop
1,2,metal
2,3,disco
3,4,blues
4,5,reggae
5,6,classical
6,7,rock
7,8,hiphop
8,9,country
9,10,jazz


In [49]:
df_class_encodings.to_csv(os.path.join(feature_store_path, 'class_encodings_{0}.csv'.format(current_time)), index=False)